In [50]:
import pandas as pd
import numpy as np
import csv
import re
import string
import nltk
from nltk.corpus import stopwords

In [51]:
url = "https://raw.githubusercontent.com/Mai4542/Content-Moderation-System/main/train.csv"
train = pd.read_csv(url,
                   on_bad_lines='skip',
                   engine='python')

In [52]:
# from google.colab import files
# uploaded = files.upload()


In [53]:
# import zipfile
# with zipfile.ZipFile("train.csv.zip", 'r') as zip_ref:
#     zip_ref.extractall("/content/")


In [54]:
# import os
# print(os.listdir("/content"))


In [55]:
# import pandas as pd
# train = pd.read_csv("/content/train.csv")
# print(train.head())


In [56]:
nltk.download('stopwords')
english_stopwords = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
train.columns = train.columns.str.replace('"', '')

In [58]:
print("First few rows:")
print(train.head())


First few rows:
                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [59]:
print("\nDataset shape:", train.shape)
print("\nColumns:", train.columns.tolist())
print("\nDataset info:")
print(train.info())


Dataset shape: (159571, 8)

Columns: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB
None


In [60]:
print("\nMissing values before processing:")
print(train.isnull().sum())


Missing values before processing:
id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [61]:
train = train.dropna(subset=['comment_text'])

In [62]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train[label_cols] = train[label_cols].fillna(0)

In [63]:
train = train.drop_duplicates()
train = train.reset_index(drop=True)

In [64]:
print("\nMissing values after processing:")
print(train.isnull().sum())


Missing values after processing:
id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


# **Text Preprocessing**

In [65]:
def clean_text(text):
    if not isinstance(text, str):
        return ""

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    text = re.sub(r'@\w+|#\w+', '', text)

    text = text.lower()

    text = text.translate(str.maketrans('', '', string.punctuation))

    text = re.sub(r'\d+', '', text)

    text = " ".join([word for word in text.split() if word not in english_stopwords])

    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [66]:
print("\nText cleaning example:")
print("Before:", train['comment_text'].iloc[1])
train['clean_comment'] = train['comment_text'].apply(clean_text)
print("After:", train['clean_comment'].iloc[1])


Text cleaning example:
Before: D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
After: daww matches background colour im seemingly stuck thanks talk january utc


# **Feature Extraction**

In [67]:
def extract_features(df):

    df['char_count'] = df['clean_comment'].apply(len)
    df['word_count'] = df['clean_comment'].apply(lambda x: len(x.split()))

    df['caps_ratio'] = df['comment_text'].apply(
        lambda x: sum(1 for c in str(x) if c.isupper()) / max(len(str(x)), 1)
    )

    df['url_count'] = df['comment_text'].apply(
        lambda x: len(re.findall(r'http\S+|www\S+', str(x)))
    )

    df['mention_count'] = df['comment_text'].apply(lambda x: str(x).count('@'))

    df['hashtag_count'] = df['comment_text'].apply(lambda x: str(x).count('#'))

    df['repeated_chars'] = df['clean_comment'].apply(
        lambda x: 1 if re.search(r'(.)\1{2,}', x) else 0
    )

    df['avg_word_length'] = df['clean_comment'].apply(
        lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0
    )

    df['space_ratio'] = df['comment_text'].apply(
        lambda x: str(x).count(' ') / max(len(str(x)), 1)
    )

    df['exclamation_count'] = df['comment_text'].apply(lambda x: str(x).count('!'))
    df['question_count'] = df['comment_text'].apply(lambda x: str(x).count('?'))

    return df

In [68]:
train = extract_features(train)

print("\nFeatures extracted:")
print(train[['clean_comment', 'char_count', 'word_count', 'caps_ratio',
             'url_count', 'mention_count', 'hashtag_count', 'repeated_chars']].head())


Features extracted:
                                       clean_comment  char_count  word_count  \
0  explanation edits made username hardcore metal...         173          26   
1  daww matches background colour im seemingly st...          73          11   
2  hey man im really trying edit war guy constant...         144          22   
3  cant make real suggestions improvement wondere...         368          49   
4                sir hero chance remember page thats          35           6   

   caps_ratio  url_count  mention_count  hashtag_count  repeated_chars  
0    0.064394          0              0              0               0  
1    0.071429          0              0              0               0  
2    0.017167          0              0              0               0  
3    0.017685          0              0              1               0  
4    0.029851          0              0              0               0  


In [69]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_comment,char_count,word_count,caps_ratio,url_count,mention_count,hashtag_count,repeated_chars,avg_word_length,space_ratio,exclamation_count,question_count
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits made username hardcore metal...,173,26,0.064394,0,0,0,0,5.692308,0.155303,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,daww matches background colour im seemingly st...,73,11,0.071429,0,0,0,0,5.727273,0.151786,1,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man im really trying edit war guy constant...,144,22,0.017167,0,0,0,0,5.590909,0.175966,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,cant make real suggestions improvement wondere...,368,49,0.017685,0,0,1,0,6.530612,0.178457,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,sir hero chance remember page thats,35,6,0.029851,0,0,0,0,5.000000,0.179104,0,1
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,congratulations well use tools well · talk,42,7,0.015385,0,0,0,0,5.142857,0.169231,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,cocksucker piss around work,27,4,0.840909,0,0,0,0,6.000000,0.159091,0,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,vandalism matt shirvington article reverted pl...,62,8,0.034783,0,0,0,0,6.875000,0.173913,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,sorry word nonsense offensive anyway im intend...,284,39,0.014831,0,0,0,0,6.307692,0.173729,0,1
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,alignment subject contrary dulithgow,36,4,0.028571,0,0,0,0,8.250000,0.157143,0,0


# **Rule-Based Filter**


In [70]:
# Rule-Based Filter System
import re
import pandas as pd

# 1. Keyword blacklists for offensive/banned terms
OFFENSIVE_KEYWORDS = [
    # Strong offensive terms
    'fuck', 'shit', 'asshole', 'bastard', 'bitch', 'cunt', 'whore', 'dick', 'pussy',
    'cock', 'motherfucker', 'crap', 'damn', 'hell', 'douche', 'slut', 'whore', 'prostitute',

    # Insults and hate speech
    'stupid', 'idiot', 'moron', 'retard', 'fool', 'dumb', 'retarded', 'imbecile',
    'hate', 'kill', 'die', 'hurt', 'attack', 'harm', 'destroy', 'murder', 'suicide',
    'loser', 'worthless', 'useless', 'trash', 'garbage', 'scum', 'jerk', 'scumbag',

    # Racial/identity slurs
    'nigger', 'nigga', 'chink', 'spic', 'kike', 'fag', 'faggot', 'dyke', 'tranny',
    'terrorist', 'nazi', 'racist', 'supremacist', 'bigot', 'homophobe', 'transphobe',

    # Sexual content
    'pedophile', 'rapist', 'molest', 'rape', 'incest', 'porn', 'xxx', 'pedo', 'childporn',

    # Add variations and common misspellings
    'fuk', 'fck', 'fuc', 'f*ck', 'f**k', 'f***', 'f****', 'f**', 'f***',
    'sh*t', 'sht', 'sh1t', 'sh!t', 's**t',
    'b*tch', 'b**ch', 'b***h', 'biatch', 'bit*h',
    'a$$', 'a55', 'a**', 'assh*le',
    'd*ck', 'd**k', 'd***',
    'p*ssy', 'p**sy', 'p***y',
    'c*nt', 'c**t', 'c***',
    'n*gger', 'n**ga', 'n***a',
    'ret*rd', 'r*tard',
    '5hit', '5hit', 'f00k', 'f00',

    # Additional common variations
    'fword', 'sword', 'thefword', 'thesword', 'fing', 'focking', 'focking'
]

In [71]:
SUSPICIOUS_PATTERNS = [
    'buy now', 'purchase here', 'click here', 'discount', 'limited time',
    'offer', 'promo code', 'cheap', 'deal', 'sale', 'order now',
    'cheapest', 'lowest price', 'make money', 'earn fast', 'work from home',
    'get rich', 'financial freedom',

    # New spam phrases
    'free followers', 'free prizes', 'free money', 'win free', 'sign up',
    'download this app', 'visit this website', 'instantly', 'limited stock'
]

In [72]:
# 3. Excessive patterns
EXCESSIVE_PATTERNS = {
    'caps': r'[A-Z]{4,}',  # 4 or more consecutive uppercase letters
    'repetition': r'(\w)\1{3,}',  # 4 or more repeated characters
    'exclamation': r'!{3,}',  # 3 or more exclamation marks
    'question': r'\?{3,}',  # 3 or more question marks
}

In [73]:
def detect_offensive_keywords_safe(text):
    """Enhanced detection that handles all variations"""
    if not isinstance(text, str):
        return 0, []

    text_lower = text.lower()
    found_keywords = []

    # Check each offensive keyword
    for keyword in OFFENSIVE_KEYWORDS:
        # Simple containment check (works for most cases)
        if keyword in text_lower:
            found_keywords.append(keyword)
            continue

        # Check with word boundaries approximation
        if f" {keyword} " in f" {text_lower} ":
            found_keywords.append(keyword)
            continue

        # Check for common separators
        for sep in ['', ' ', '.', ',', '!', '?', '*', '-', '_']:
            if f"{sep}{keyword}{sep}" in text_lower:
                found_keywords.append(keyword)
                break

    return len(found_keywords), found_keywords

In [74]:
def detect_suspicious_patterns_safe(text):
    """Detect suspicious promotional patterns without regex"""
    if not isinstance(text, str):
        return 0, []

    text_lower = text.lower()
    found_patterns = []

    for pattern in SUSPICIOUS_PATTERNS:
        if pattern in text_lower:
            found_patterns.append(pattern)

    return len(found_patterns), found_patterns

In [75]:
def detect_excessive_patterns_safe(text):
    """Detect excessive patterns without any regex"""
    if not isinstance(text, str):
        return {}

    results = {
        'excessive_caps': 0,
        'character_repetition': 0,
        'excessive_exclamation': 0,
        'excessive_question': 0
    }

    # Check for excessive caps (4+ consecutive uppercase)
    consecutive_caps = 0
    for char in text:
        if char.isupper():
            consecutive_caps += 1
            if consecutive_caps >= 4:
                results['excessive_caps'] += 1
        else:
            consecutive_caps = 0

    # Check for character repetition (4+ same character in a row)
    for i in range(len(text) - 3):
        if text[i] == text[i+1] == text[i+2] == text[i+3] and text[i].isalnum():
            results['character_repetition'] += 1

    # Check for excessive punctuation
    consecutive_excl = 0
    for char in text:
        if char == '!':
            consecutive_excl += 1
            if consecutive_excl >= 3:
                results['excessive_exclamation'] += 1
        else:
            consecutive_excl = 0

    consecutive_quest = 0
    for char in text:
        if char == '?':
            consecutive_quest += 1
            if consecutive_quest >= 3:
                results['excessive_question'] += 1
        else:
            consecutive_quest = 0

    return results

In [76]:
def calculate_risk_score(keyword_count, pattern_count, excessive_results):
    """Calculate overall risk score based on all factors"""
    # Weighted scoring
    score = (
        min(keyword_count * 0.3, 1.0) +  # Max 1.0 for keywords
        min(pattern_count * 0.2, 0.5) +   # Max 0.5 for patterns
        min(excessive_results.get('excessive_caps', 0) * 0.1, 0.2) +
        min(excessive_results.get('character_repetition', 0) * 0.1, 0.2) +
        min(excessive_results.get('excessive_exclamation', 0) * 0.05, 0.05) +
        min(excessive_results.get('excessive_question', 0) * 0.05, 0.05)
    )

    return min(score, 1.0)  # Cap at 1.0

In [77]:
def rule_based_filter_safe(text):
    """Main rule-based filter function - COMPLETELY REGEX-FREE"""
    if not isinstance(text, str):
        return {
            'offensive_keywords_count': 0,
            'offensive_keywords_found': [],
            'suspicious_patterns_count': 0,
            'suspicious_patterns_found': [],
            'excessive_patterns': {},
            'risk_score': 0,
            'risk_level': 'low',
            'action': 'safe'
        }

    # Detect offensive keywords
    keyword_count, found_keywords = detect_offensive_keywords_safe(text)

    # Detect suspicious patterns
    pattern_count, found_patterns = detect_suspicious_patterns_safe(text)

    # Detect excessive patterns using safe method
    excessive_results = detect_excessive_patterns_safe(text)

    # Calculate risk score
    risk_score = calculate_risk_score(keyword_count, pattern_count, excessive_results)

    # Determine action
    if pattern_count > 0:
        action = 'spam'
    elif risk_score >= 0.7:
        action = 'high_risk'
    elif risk_score >= 0.4:
        action = 'medium_risk'
    else:
        action = 'safe'

    # Map action to risk_level for backward compatibility
    if action == 'high_risk':
        risk_level = 'high'
    elif action == 'medium_risk':
        risk_level = 'medium'
    else:
        risk_level = 'low'

    return {
        'offensive_keywords_count': keyword_count,
        'offensive_keywords_found': found_keywords,
        'suspicious_patterns_count': pattern_count,
        'suspicious_patterns_found': found_patterns,
        'excessive_patterns': excessive_results,
        'risk_score': risk_score,
        'risk_level': risk_level,
        'action': action
    }

# Apply the filter
print("Applying completely regex-free rule-based filter to comments...")

sample_size = min(1000, len(train))
train_sample = train.head(sample_size).copy()

filter_results = train_sample['comment_text'].apply(rule_based_filter_safe)
print("Filter applied successfully!")

# Convert results to DataFrame and merge with original data
filter_results_df = pd.DataFrame(filter_results.tolist())
train_with_filter = pd.concat([train_sample, filter_results_df], axis=1)

print("\nRule-based filter results:")
print(train_with_filter[['comment_text', 'risk_score', 'risk_level', 'action',
                         'offensive_keywords_count', 'suspicious_patterns_count']].head(10))

Applying completely regex-free rule-based filter to comments...
Filter applied successfully!

Rule-based filter results:
                                        comment_text  risk_score risk_level  \
0  Explanation\nWhy the edits made under my usern...         0.0        low   
1  D'aww! He matches this background colour I'm s...         0.0        low   
2  Hey man, I'm really not trying to edit war. It...         0.0        low   
3  "\nMore\nI can't make any real suggestions on ...         0.0        low   
4  You, sir, are my hero. Any chance you remember...         0.0        low   
5  "\n\nCongratulations from me as well, use the ...         0.0        low   
6       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK         0.5     medium   
7  Your vandalism to the Matt Shirvington article...         0.0        low   
8  Sorry if the word 'nonsense' was offensive to ...         0.0        low   
9  alignment on this subject and which are contra...         0.0        low   

        a

In [78]:
# Display results
print("\nRule-based filter results:")
print(train_with_filter[['comment_text', 'risk_score', 'risk_level',
                         'offensive_keywords_count', 'suspicious_patterns_count']].head(10))

# Summary statistics
print("\nRisk Level Distribution:")
print(train_with_filter['risk_level'].value_counts())

print("\nAverage risk score by toxicity label:")
if 'toxic' in train_with_filter.columns:
    print(train_with_filter.groupby('toxic')['risk_score'].mean())


Rule-based filter results:
                                        comment_text  risk_score risk_level  \
0  Explanation\nWhy the edits made under my usern...         0.0        low   
1  D'aww! He matches this background colour I'm s...         0.0        low   
2  Hey man, I'm really not trying to edit war. It...         0.0        low   
3  "\nMore\nI can't make any real suggestions on ...         0.0        low   
4  You, sir, are my hero. Any chance you remember...         0.0        low   
5  "\n\nCongratulations from me as well, use the ...         0.0        low   
6       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK         0.5     medium   
7  Your vandalism to the Matt Shirvington article...         0.0        low   
8  Sorry if the word 'nonsense' was offensive to ...         0.0        low   
9  alignment on this subject and which are contra...         0.0        low   

   offensive_keywords_count  suspicious_patterns_count  
0                         0                  

In [79]:
# Save the results with rule-based filtering
output_path = "/content/sample_data/train_with_rule_filter.csv"
train_with_filter.to_csv(output_path, index=False)
print(f"\nData with rule-based filter saved to: {output_path}")


Data with rule-based filter saved to: /content/sample_data/train_with_rule_filter.csv


# **Machine Learning** **Classifier**

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack,csr_matrix
x_text=train['clean_comment']
x_features=train[['char_count', 'word_count', 'caps_ratio',
             'url_count', 'mention_count', 'hashtag_count', 'repeated_chars','avg_word_length','space_ratio','exclamation_count','question_count']]
x_features=csr_matrix(x_features.values)
vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
X_text_tfidf = vectorizer.fit_transform(x_text)
X = hstack([X_text_tfidf, x_features])
y=train[['toxic', 'severe_toxic', 'obscene', 'threat',
             'insult', 'identity_hate']]

In [81]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [82]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import classification_report
# model = OneVsRestClassifier(LogisticRegression(max_iter=3000))
# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)
# print(classification_report(y_test, y_pred, target_names=y.columns))

In [83]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.multiclass import OneVsRestClassifier
# model_nb = OneVsRestClassifier(MultinomialNB())
# model_nb.fit(x_train, y_train)
# y_pred_nb = model_nb.predict(x_test)
# print(classification_report(y_test, y_pred_nb, target_names=y.columns))
# # NOOOOOOOO

In [84]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import classification_report
# model_rf = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
# model_rf.fit(x_train, y_train)
# y_pred_rf = model_rf.predict(x_test)
# print(classification_report(y_test, y_pred_rf, target_names=y.columns))
# #NOOOOOOO

In [85]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report,accuracy_score
model_svm = OneVsRestClassifier(LinearSVC(class_weight="balanced", max_iter=5000))
model_svm.fit(x_train, y_train)
y_pred_svm = model_svm.predict(x_test)
print(classification_report(y_test, y_pred_svm, target_names=y.columns))
#THE RIGHT MODEL



               precision    recall  f1-score   support

        toxic       0.57      0.83      0.67      3056
 severe_toxic       0.24      0.79      0.37       321
      obscene       0.56      0.86      0.68      1715
       threat       0.14      0.64      0.23        74
       insult       0.44      0.84      0.58      1614
identity_hate       0.17      0.66      0.27       294

    micro avg       0.46      0.83      0.59      7074
    macro avg       0.35      0.77      0.47      7074
 weighted avg       0.50      0.83      0.62      7074
  samples avg       0.06      0.08      0.06      7074



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [86]:
from scipy.sparse import hstack, csr_matrix
def prepare_row_for_model(row, vectorizer):
    text = row['clean_comment']
    X_text = vectorizer.transform([text])
    num_features = ['char_count', 'word_count', 'caps_ratio',
                    'url_count', 'mention_count', 'hashtag_count',
                    'repeated_chars', 'avg_word_length',
                    'space_ratio', 'exclamation_count', 'question_count']
    x_features = row[num_features].astype(float).values.reshape(1, -1)
    x_features = csr_matrix(x_features)
    return hstack([X_text, x_features])

In [87]:
row = train.iloc[6]
X_row = prepare_row_for_model(row, vectorizer)
pred = model_svm.predict(X_row)
print(pred)

[[1 1 1 0 1 0]]


In [88]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
def interpret_prediction(pred):
    pred = pred[0]
    toxic_related = {"toxic", "severe_toxic", "obscene", "insult", "identity_hate"}
    spam_related = {"threat"}
    # multi-labels detected
    active_labels = [labels[i] for i, val in enumerate(pred) if val == 1]
    if active_labels:
        print("Detected labels:", ", ".join(active_labels))
    else:
        print("Detected labels: None")
    # final result
    if any(label in spam_related for label in active_labels):
        print("Final Result: spam")
    elif any(label in toxic_related for label in active_labels):
        print("Final Result: toxic")
    else:
        print("Final Result: safe")



In [89]:
def extract_features_single(comment, clean_comment):
    return [
        len(clean_comment),                             # char_count
        len(clean_comment.split()),                     # word_count
        sum(1 for c in comment if c.isupper()) / max(len(comment), 1),  # caps_ratio
        len(re.findall(r'http\S+|www\S+', comment)),    # url_count
        comment.count('@'),                             # mention_count
        comment.count('#'),                             # hashtag_count
        1 if re.search(r'(.)\1{2,}', clean_comment) else 0,  # repeated_chars
        np.mean([len(w) for w in clean_comment.split()]) if clean_comment.split() else 0,  # avg_word_length
        comment.count(' ') / max(len(comment), 1),      # space_ratio
        comment.count('!'),                             # exclamation_count
        comment.count('?')                              # question_count
    ]
from scipy.sparse import csr_matrix, hstack

def prepare_single_comment(comment, vectorizer):
    clean_comment = clean_text(comment)
    comment_tfidf = vectorizer.transform([clean_comment])
    features = extract_features_single(comment, clean_comment)
    features_sparse = csr_matrix([features])  # 2D
    final_input = hstack([comment_tfidf, features_sparse])
    return final_input


In [90]:
pre_comment="COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK"
comment=prepare_single_comment(pre_comment,vectorizer)
pred = model_svm.predict(comment)
print(pred)
interpret_prediction(pred)

[[1 1 1 0 1 0]]
Detected labels: toxic, severe_toxic, obscene, insult
Final Result: toxic


In [91]:
acc = accuracy_score(y_test, y_pred_svm)
acc

0.8360645464515118

In [92]:
import joblib
joblib.dump(model_svm, "content_moderation_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")


['vectorizer.pkl']

# RiskAssessmentEngine

In [93]:
# RiskAssessmentEngine - MODIFIED FOR SPAM DETECTION
!pip install gradio
import gradio as gr
import numpy as np
from scipy.sparse import hstack, csr_matrix

def extract_features_single_fixed(comment):
    clean_comment = clean_text(comment)
    return [
        len(clean_comment),
        len(clean_comment.split()),
        sum(1 for c in comment if c.isupper()) / max(len(comment), 1),
        comment.count('http'),
        comment.count('@'),
        comment.count('#'),
        0,
        np.mean([len(w) for w in clean_comment.split()]) if clean_comment.split() else 0,
        comment.count(' ') / max(len(comment), 1),
        comment.count('!'),
        comment.count('?')
    ]

def prepare_single_comment_fixed(comment, vectorizer):
    clean_comment_text = clean_text(comment)
    comment_tfidf = vectorizer.transform([clean_comment_text])
    features = extract_features_single_fixed(comment)
    features_sparse = csr_matrix([features])
    return hstack([comment_tfidf, features_sparse])

class RiskAssessmentEngine:
    def __init__(self, model, vectorizer, filter_func, weights=None, thresholds=None):
        self.model = model
        self.vectorizer = vectorizer
        self.filter_func = filter_func
        self.weights = weights or {"rule": 0.7, "ml": 0.3}
        self.thresholds = thresholds or {"block": 0.6, "review": 0.4}

    def score_text(self, text, for_user=False):
        # 1. Rule-based filter
        rule_result = self.filter_func(text)
        rule_score = rule_result.get("risk_score", 0)
        suspicious_patterns_count = rule_result.get("suspicious_patterns_count", 0)

        # 2. ML model prediction
        try:
            X = prepare_single_comment_fixed(text, self.vectorizer)
            ml_prediction = self.model.predict(X)[0]
            ml_score = 0.8 if any(ml_prediction) else 0.1
        except Exception as e:
            print(f"ML prediction error: {e}")
            ml_score = 0.1
            ml_prediction = [0] * 6

        # 3. Weighted combination
        combined = 0.7 * rule_score + 0.3 * ml_score if not any(ml_prediction) else 0.3 * rule_score + 0.7 * ml_score

        # 4. Action decision
        threat_keywords = ["destroy", "kill", "die", "attack", "hurt", "murder"]
        has_threat = any(keyword in text.lower() for keyword in threat_keywords)

        if suspicious_patterns_count > 0:
            action = "spam"
        elif has_threat or any(ml_prediction):
            action = "toxic"
        elif rule_score >= self.thresholds["block"] or combined >= self.thresholds["block"]:
            action = "toxic"
        elif combined >= self.thresholds["review"]:
            action = "toxic"
        else:
            action = "safe"

        if for_user:
            return action
        else:
            return {
                "text": text,
                "rule_score": round(rule_score, 3),
                "ml_score": round(ml_score, 3),
                "combined_score": round(combined, 3),
                "action": action,
                "ml_prediction": ml_prediction.tolist() if hasattr(ml_prediction, 'tolist') else ml_prediction
            }

# Create fixed engine
engine = RiskAssessmentEngine(
    model=model_svm,
    vectorizer=vectorizer,
    filter_func=rule_based_filter_safe
)

# Test examples
test_texts = [
    "buy now and get free prizes",
    "I will kill you",
    "This is a normal comment"
]

print("Testing fixed engine:")
for text in test_texts:
    result = engine.score_text(text)
    print(f"Text: '{text}' -> {result['action']} (rule: {result['rule_score']}, ml: {result['ml_score']}, combined: {result['combined_score']})")

# Gradio interface
def predict_comment(comment):
    return engine.score_text(comment, for_user=True)

demo = gr.Interface(
    fn=predict_comment,
    inputs=gr.Textbox(lines=3, placeholder="Enter a comment..."),
    outputs="text",
    title="Content Moderation Checker",
    description="Classify comments into toxic, spam, or safe.",
    allow_flagging="never"
)

demo.launch()

Testing fixed engine:
Text: 'buy now and get free prizes' -> spam (rule: 0.4, ml: 0.8, combined: 0.68)
Text: 'I will kill you' -> toxic (rule: 0.3, ml: 0.8, combined: 0.65)
Text: 'This is a normal comment' -> toxic (rule: 0.0, ml: 0.8, combined: 0.56)


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12c6762449cb5745a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
